In [1]:
import sys
sys.path.append('src')

In [2]:
import locale
locale.setlocale(locale.LC_TIME, 'fr_FR')   ## set French as local language, useful for date display
# locale.setlocale(locale.LC_TIME, 'fr_FR.UTF-8')  # non-windows computer? 

import pandas as pd

from models.telegram import TelegramModel
from models.messenger import MessengerModel
from utils.formatting import process_for_latex
from utils.timing import add_timing_to_df
from utils.formatting import left_formating, right_formating, left_formating_with_bubbles
from utils.formatting import right_formating_with_bubbles, format_msg

In [3]:
telegram_model = TelegramModel()
messenger_model = MessengerModel()

In [4]:
telegram_data_path = 'data/telegram/ChatExport_2020-11-15/result.json'
concatenated_table_telegram = telegram_model.parse_from_json(telegram_data_path)
concatenated_table_telegram.info()
concatenated_table_telegram.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   source    13 non-null     object
 1   datetime  13 non-null     object
 2   sender    13 non-null     object
 3   message   13 non-null     object
 4   path      13 non-null     object
dtypes: object(5)
memory usage: 648.0+ bytes


,source,datetime,sender,message,path
0,Telegram,2020-06-16T18:01:09,B,La pandémie du covid-19 est une maladie nouvel...,[]
1,Telegram,2020-06-16T18:06:17,M,Lois mathématiques --,[]
2,Telegram,2020-06-17T09:19:55,M,"J'ai lu le teasing d'intro, super bien posé, h...",[]
3,Telegram,2020-07-07T15:54:05,B,Du coup si on résume : \ * 🚂 Départ J-0 à 20h5...,[]
4,Telegram,2020-07-18T00:16:11,B,j'ai meme fait le graphique de la médecin,photos/photo_1@18-07-2020_00-16-11.jpg


In [5]:
messenger_data_path = 'data/messenger/message_1.json'
concatenated_table_messenger = messenger_model.parse_from_json(messenger_data_path)
concatenated_table_messenger.shape

https://interncache-frc.fbcdn.net/v/t35.18174-12/28034952_10214165916714647_1547945972_o.jpg?ccb=2&_nc_sid=73a6a0&efg=eyJ1cmxnZW4iOiJwaHBfdXJsZ2VuX2NsaWVudC9pbW9nZW46RFlJTWVkaWFVdGlscyJ9&_nc_ad=z-m&_nc_cid=0&_nc_ht=interncache-frc&oh=581344d64b50047f0a43787643d161ae&oe=5FC53790 <re.Match object; span=(49, 92), match='28034952_10214165916714647_1547945972_o.jpg'>


(55, 5)

In [6]:
concatenated_table_messenger['path'].tolist()


[[],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['data/messenger/photos/28034952_10214165916714647_1547945972_o_10214165916714647.jpg'],
 [],
 [],
 [],
 [],
 [],
 []]

In [7]:
# Merge two dataframes
concatenated_table_messenger = concatenated_table_messenger.append(concatenated_table_telegram)
concatenated_table_messenger.shape

(68, 5)

In [9]:
concatenated_table_messenger = add_timing_to_df(concatenated_table_messenger)

In [10]:
# Other option to print the tex with one message per bubble.

df = concatenated_table_messenger
separate_messages = True

for date, df_t in df.groupby('date'):
    date_str = df_t['dateStr'].unique()[0]
    title = '\\section*{' + date_str + '}'
    
    if separate_messages:
        discussion = '\n'.join(df_t.groupby('message').apply(format_msg))
    else:
        discussion = '\n'.join(df_t.groupby('senderGroup').apply(format_msg))
    text = f'{title}\n{discussion}'
    
    with open(f'./sections/{date}.tex', 'w', encoding='utf-8') as ft:
        ft.write(text)



FileNotFoundError: [Errno 2] No such file or directory: './sections/2018-06-30.tex'

In [117]:
# Loop over dates, generate yyyy-mm-dd.tex section files, one per day
# -------------------------------------------------------------------
# Maud version <3

df = concatenated_table


separate_messages = True

for date, df_t in df.groupby('date'):
    date_str = df_t['dateStr'].unique()[0]
    title = '\\section*{' + date_str + '}'
    
    if separate_messages:
        discussion = ' '.join(df_t.groupby('senderGroup').apply(format_msg))
    else:
        discussion = '\n\n'.join(df_t.groupby('senderGroup').apply(format_msg))
    
    text = f'{title}\n\n{discussion}'
    
    with open(f'./sections/{date}.tex', 'w', encoding='utf-8') as ft:
        ft.write(text)